1. 图像和文本特征提取
使用图像编码器（如 ResNet 或视觉 Transformer）对图像进行编码，得到图像特征向量I_f
使用文本编码器（如 CBOW 或文本 Transformer）对文本进行编码，得到文本特征向量T_f

In [1]:
import torch
import torch.nn as nn
import torchvision.models as models
from transformers import BertModel, BertTokenizer

# 图像编码器
class ImageEncoder(nn.Module):
    def __init__(self):
        super(ImageEncoder, self).__init__()
        self.resnet = models.resnet50(pretrained=True)
        self.resnet.fc = nn.Identity()  # 去掉全连接层

    def forward(self, x):
        return self.resnet(x)

# 文本编码器
class TextEncoder(nn.Module):
    def __init__(self):
        super(TextEncoder, self).__init__()
        self.bert = BertModel.from_pretrained('bert-base-uncased')

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        return outputs.last_hidden_state[:, 0, :]  # 取 [CLS] 标记的嵌入

# 示例数据
image_encoder = ImageEncoder()
text_encoder = TextEncoder()

# 图像数据
image_data = torch.randn(16, 3, 224, 224)  # 16 张 224x224 的图像
image_features = image_encoder(image_data)

# 文本数据
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
texts = ["This is a sample text."] * 16
input_ids = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')['input_ids']
attention_mask = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')['attention_mask']
text_features = text_encoder(input_ids, attention_mask)

D:\soft\New\miniconda\envs\torch-play\lib\site-packages\torchvision\models\_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
D:\soft\New\miniconda\envs\torch-play\lib\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


2. 投影到共同嵌入空间
将图像特征I_f和文本特征T_f投影到嵌入空间，并进行 L2 归一化。

In [2]:
class ProjectionHead(nn.Module):
    def __init__(self, input_dim, output_dim):
        super(ProjectionHead, self).__init__()
        self.fc = nn.Linear(input_dim, output_dim)

    def forward(self, x):
        return self.fc(x)

# 投影头
image_projection_head = ProjectionHead(image_features.shape[1], 512) # 投影头是一个全连接层（线性层），它将输入特征的维度映射到指定的输出维度。投影头将输入特征的维度 input_dim 映射到 512 维的嵌入空间。投影头的作用是将高维的图像特征和文本特征映射到一个低维的嵌入空间，以便进行相似度计算。通过投影头，可以将不同模态（图像和文本）的特征映射到同一个嵌入空间，从而方便计算它们之间的相似度。
text_projection_head = ProjectionHead(text_features.shape[1], 512)

# 投影并归一化
image_embeddings = nn.functional.normalize(image_projection_head(image_features), dim=1) # 使用投影头将图像特征和文本特征投影到嵌入空间，并进行 L2 归一化。使用 nn.functional.normalize 函数对投影后的特征进行 L2 归一化，使得每个特征向量的 L2 范数为 1。L2 归一化可以统一特征向量的尺度，提高相似度计算的稳定性和模型的泛化能力
text_embeddings = nn.functional.normalize(text_projection_head(text_features), dim=1)

3. 计算相似性分数
通过计算图像和文本嵌入的余弦相似度，得到相似性矩阵。

In [3]:
import torch.nn.functional as F

# 计算相似性矩阵
logits = torch.matmul(image_embeddings, text_embeddings.T)
logits = logits * torch.exp(torch.tensor(1.0))  # 温度参数 t

4. 构建损失函数
使用交叉熵损失分别对图像和文本进行监督。

In [1]:
# 标签
labels = torch.arange(logits.shape[0])

# 对图像的损失
loss_i = F.cross_entropy(logits, labels) # 这里的损失不仅包括正确匹配的图像和文本对的损失，还包括不匹配的图像和文本对的损失。nn.CrossEntropyLoss 的输入是未归一化的 logits（即未经过 softmax 的输出），而不是经过 softmax 的概率分布。nn.CrossEntropyLoss 内部会自动对 logits 进行 softmax 操作，然后计算交叉熵损失。

# 对文本的损失
loss_t = F.cross_entropy(logits.T, labels)

# 最终损失
loss = (loss_i + loss_t) / 2 # 为什么求这两部分的平均值：通过计算两部分损失的平均值，可以平衡图像和文本的匹配情况，使得模型在图像和文本的匹配任务上都能得到良好的表现

NameError: name 'torch' is not defined

## 在深度学习中，logits 和 softmax 是两个密切相关的概念，尤其是在分类任务中。
1. Logits
定义: logits 是指神经网络最后一层（通常是全连接层）输出的原始分数（未经过任何激活函数处理的值）。
形状: 对于一个分类问题，假设有 C 个类别，logits 的形状通常是 (batch_size, C)，其中 batch_size 是输入样本的数量，C 是类别的数量。
特点: logits 是未归一化的值，表示模型对每个类别的“信心”或“分数”。这些分数可以是任意实数，不限于 [0, 1] 区间。
2. Softmax
定义: softmax 是一种激活函数，用于将 logits 转换为概率分布。softmax 函数的输出是一个概率分布，表示模型对每个类别的预测概率。
公式: 对于一个样本的 logits 向量 z = [z1, z2, ..., zC]，softmax 的输出 p = [p1, p2, ..., pC] 计算如下：
其中 e 是自然对数的底数。
特点: softmax 的输出是一个概率分布，即所有 pi 的和为 1，并且每个 pi 都在 [0, 1] 区间内。
3. 关系
顺序: logits 是 softmax 的输入，softmax 是 logits 的输出。
用途: 在分类任务中，logits 通常作为 softmax 的输入，softmax 的输出用于计算交叉熵损失（如 nn.CrossEntropyLoss）。
计算: nn.CrossEntropyLoss 内部会自动对 logits 进行 softmax 操作，然后计算交叉熵损失。因此，在使用 nn.CrossEntropyLoss 时，直接传入 logits 即可，不需要手动进行 softmax 操作。

5. 优化目标
最大化对角线元素的相似性（匹配的图像-文本对），最小化非对角线元素的相似性（不匹配的图像-文本对）。

In [5]:
# 优化器
optimizer = torch.optim.Adam([
    {'params': image_encoder.parameters()},
    {'params': text_encoder.parameters()},
    {'params': image_projection_head.parameters()},
    {'params': text_projection_head.parameters()}
], lr=1e-4)

# 反向传播和优化
optimizer.zero_grad()
loss.backward()
optimizer.step()

In [6]:
print(f"Total Loss: {loss.item()}")

Total Loss: 2.773303985595703


总结
CLIP 模型的训练过程可以总结为以下几个步骤：
1. 图像和文本特征提取：
- 使用图像编码器对图像进行编码，得到图像特征向量 
- 使用文本编码器对文本进行编码，得到文本特征向量 
2. 投影到共同嵌入空间：
- 将图像特征和文本特征投影到嵌入空间，并进行 L2 归一化。
3. 计算相似性分数：
- 通过计算图像和文本嵌入的余弦相似度，得到相似性矩阵。
4. 构建损失函数：
- 使用交叉熵损失分别对图像和文本进行监督。
- 最终损失为两者的平均。
5. 优化目标：
- 最大化对角线元素的相似性（匹配的图像-文本对）。
- 最小化非对角线元素的相似性（不匹配的图像-文本对）。

通过这些步骤，可以实现 CLIP 模型的训练过程，使得在嵌入空间中，正确匹配的图像和文本对的相似度更高。